In [0]:
!pip install tweet-preprocessor
!pip install google-api-python-client

import pandas as pd
import preprocessor as p
import networkx as nx
import matplotlib.pyplot as plt
import tweepy
import json
import json
import requests
import csv
import re
import sys
import urllib.parse

from sklearn.feature_extraction.text import CountVectorizer

from googleapiclient.discovery import build
from datetime import datetime

In [0]:
def decode_title(dct):
        if "title" in dct:
            return dct["title"]     
        return dct
def decode_date(dct):
        if "publishedAt" in dct:
            return dct["publishedAt"]    
        return dct
def decode_source(dct):
        if "source" in dct:
            return dct["source"]    
        return dct
def decode_url(dct):
        if "url" in dct:
            return dct["url"]   
        return dct
def decode_description(dct):
        if "description" in dct:
            return p.clean(dct["description"])      
        return dct

In [0]:
import requests
from bs4 import BeautifulSoup
import spacy
ner = spacy.load("en_core_web_sm")
def getArticle(url):
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)

    article_text = ''
    article = soup.findAll('p')
    for element in article:
        article_text += '\n' + ''.join(element.findAll(text = True))
    return article_text

def getEntity(article_cor):
  ent = ner(article_cor)
  entities = []
  for entity in ent.ents:
    entities.append(entity.text)
  return entities

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]

def getTopics(url):
    topic = ""
    article_text = getArticle(url)
    articleEntity = getEntity(article_text)
    topics = get_top_n_words(articleEntity,10)
    for item in topics:
      topic = topic + item[0] + " "
    return topic.strip()


In [0]:
from datetime import datetime
from pytz import timezone
import pytz
date_format='%m/%d/%Y %H:%M:%S %Z'
date = datetime.now(tz=pytz.utc)
date = date.astimezone(timezone('US/Pacific'))
print(date)
time = date.strftime("%m/%d/%Y-%H-%M")
timestamp = time.replace("/","-")
print("time:", timestamp)

In [0]:
# publishedAt name(source) url description

def extract_google_news_data(query):
    df = pd.DataFrame()
    keywords = urllib.parse.quote(query)
    url = "https://gnews.io/api/v3/search?q="+keywords+"&token=a6419076f909fc74c42016c6bebf0755"
    response = requests.get(url)
    gNewsApi_json = json.dumps(response.json(), indent = 4,sort_keys=True)
    parsed_title = json.loads(gNewsApi_json,object_hook=decode_title)
    parsed_date = json.loads(gNewsApi_json,object_hook=decode_date)
    parsed_source = json.loads(gNewsApi_json,object_hook=decode_source)
    parsed_url = json.loads(gNewsApi_json,object_hook=decode_url)
    parsed_description = json.loads(gNewsApi_json,object_hook=decode_description)
    date = parsed_date["articles"]
    date = [i[:10] for i in date]
    headline = parsed_title["articles"]
    source = [i["name"] for i in parsed_source["articles"]]
    url = parsed_url["articles"]
    description = parsed_description["articles"]
    # topics = [getTopics(i) for i in url]
    df["DATE"] = date
    df["HEADLINE"] = headline
    df["SOURCE"] = source
    df["URL"] = url
    df["DESCRIPTION"] = description
    df["API"] = "GOOGLE NEWS API"
    # df["TOPICS"] = topics
    filename = "gnews-"+timestamp+".csv"
    print(df)
    df.to_csv(filename)
    return df


# title = parsed_title["articles"]
# keywords = urllib.parse.quote(query)
# url = "https://newsapi.org/v2/everything?q="+keywords+"&from=&apiKey=2ed8bf16847a458d9af4ed36ce4f979b"
# response = requests.get(url)
# newsApi_json = json.dumps(response.json(),indent = 4,sort_keys=True)
# print(newsApi_json)

In [0]:
  def extract_news_api_data(query):
      df = pd.DataFrame()
      keywords = urllib.parse.quote(query)
      url = "https://newsapi.org/v2/everything?q="+keywords+"&from=&apiKey=2ed8bf16847a458d9af4ed36ce4f979b"
      response = requests.get(url)
      newsApi_json = json.dumps(response.json(),indent=4,sort_keys=True)
      # print(newsApi_json)
      parsed_title = json.loads(newsApi_json,object_hook=decode_title)
      parsed_date = json.loads(newsApi_json,object_hook=decode_date)
      parsed_source = json.loads(newsApi_json,object_hook=decode_source)
      parsed_url = json.loads(newsApi_json,object_hook=decode_url)
      parsed_description = json.loads(newsApi_json,object_hook=decode_description)
      date = parsed_date["articles"]
      date = [i[:10] for i in date]
      headline = parsed_title["articles"]
      source = [i["name"] for i in parsed_source["articles"]]
      url = parsed_url["articles"]
      description = parsed_description["articles"]
      df["DATE"] = date
      df["HEADLINE"] = headline
      df["SOURCE"] = source
      df["URL"] = url
      df["DESCRIPTION"] = description
      df["API"] = "NEWS API"
      # df["TOPICS"] = topics
      filename = "newsapi-"+timestamp+".csv"
      print(df)
      df.to_csv(filename)
      return df


In [0]:
query = "coronavirus"
df_gnews = extract_google_news_data(query)
df_newsapi = extract_news_api_data(query)

result = df_gnews.append(df_newsapi,ignore_index=True, sort=False)
filename = "news-"+timestamp+".csv"
result.to_csv(filename,index=False)